In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/clase_11/clase_11/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Intro

En esta parte vimos algunos métodos de Pandas para calcular medidas por grupos.

Para los ejercicios usaremos un dataset de inmuebles en venta en la Ciudad de Buenos Aires en el año 2016 que disponibiliza el GCBA.

La idea de esta práctica es repasar algunos temas que vimos en clases anteriores, y ejercitar los distintos métodos que vimos en ésta y vamos a volver a usar en la clase presencial.


## Dataset

El Gobierno de la Ciudad de Buenos Aires disponibiliza algunos datasets para acceso público en la url 
https://data.buenosaires.gob.ar/

Allí encontramos datos de propiedades inmuebles en venta, separados por año
https://data.buenosaires.gob.ar/dataset/departamentos-venta

En esta clase vamos a usar un subconjunto de los datos de "Departamentos en venta 2016" que descargamos desde aquí
https://data.buenosaires.gob.ar/dataset/departamentos-venta/archivo/juqdkmgo-7031-resource

En la carpeta Data de esta clase ya tenemos descargardo ese dataset, el nombre del archivo es **departamentos-en-venta-2016.csv**.

## Imports

In [2]:
import pandas as pd
import numpy as np

## Ejercicio 1  - Importar 

Leamos los datos del archivo departamentos-en-venta-2016.csv

Veamos cuántos registros tiene y de qué tipos son los datos de cada columna. 

Veamos los primeros registros para verificar que los datos fueron importados correctamente.

In [3]:
data_location = "../Data/departamentos-en-venta-2016.csv"
data = pd.read_csv(data_location, sep=';')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7564 entries, 0 to 7563
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CALLE                    6182 non-null   object 
 1   NUMERO                   6182 non-null   float64
 2   ID_ZONAPRO               7564 non-null   int64  
 3   OPERACION                7564 non-null   object 
 4   TIPO                     7564 non-null   object 
 5   M2                       7564 non-null   int64  
 6   M2CUB                    7564 non-null   int64  
 7   PRECIOTEXT               7562 non-null   object 
 8   PRECIOARS                7564 non-null   int64  
 9   PRECIOARSM               7564 non-null   int64  
 10  DOLARES                  7564 non-null   int64  
 11  U_S_M2                   7564 non-null   int64  
 12  AMBIENTES                7564 non-null   int64  
 13  ANTIGUEDAD               7564 non-null   int64  
 14  BAÑOS                   

In [4]:
data.head(3)

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,URL,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42408...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,HTTP://WWW.ZONAPROP.COM.AR/PROPIEDADES/A-42518...,NaN,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609


## Ejercicio 2  - Limpieza

Vamos a crear una columan nueva (PRECIOTEXT_CLEAN) de tipo numérico, que tenga los valores en dolares que leemos en el campo PRECIOTEXT, o NaN si esos valores están expresados en pesos.

Para eso vamos a:

1) Crear un objeto Series que resulte de limpiar los valores del campo PRECIOTEXT. Tenemos que eliminar los símbolos "U\\$S" y "."

2) En la instancia de Series resultado del punto 1) reemplazamos los valores en pesos ('\\$') por nulos (para que nos ensucien los datos de promedio). 

3) Crear una columna nueva de tipo numérico en el DataFrame (PRECIOTEXT_CLEAN) y asignar los valores de resultado de 2)


In [5]:
df = data.copy()

In [6]:
df.PRECIOTEXT.sample(15)

6750    U$S 170.000
817     U$S 206.000
5155    U$S 136.171
3135    $ 1.488.930
6089    U$S 122.125
5092     U$S 58.000
2136    U$S 140.000
934     U$S 250.000
7107     U$S 85.117
3379     U$S 76.559
3241     U$S 89.000
4476    U$S 161.000
1661    U$S 109.000
2004    U$S 105.000
851     U$S 166.000
Name: PRECIOTEXT, dtype: object

In [7]:
import re

patron_usd = re.compile(r'U\$S') 

patron_dot = re.compile(r'\.')

sin_usd = df.PRECIOTEXT.apply(lambda x : x if x is np.NaN else patron_usd.sub("", x))

sin_usd_dot = sin_usd.apply(lambda y : y if y is np.NaN else patron_dot.sub("", y))


#Ahora me creo una mascara booleana para eliminar las filas que tengan pesos

patron_peso = re.compile(r'\$')

fila_pesos = sin_usd_dot.apply(lambda x: x if x is np.NaN else re.match(patron_peso, x))

mascara_pesos = fila_pesos.notnull()

sin_usd_dot.loc[mascara_pesos] = np.NaN


df['PRECIOTEXTCLEAN']=sin_usd_dot.astype(float)

In [8]:
df.sample(5)

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,REVISION,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD,PRECIOTEXTCLEAN
1244,ITUZAINGO,925.0,41855878,VTA,DTO,63,55,U$S 149.058,2608515,0,...,NaN,NaN,ITUZAINGO 925,BARRACAS,COMUNA 04,1272.0,C1272AFE,-34.632252,-58.376755,149058.0
6708,DOBLAS,955.0,41964233,VTA,DTO,41,41,U$S 109.453,1915427,0,...,NaN,NaN,DOBLAS 955,PARQUE CHACABUCO,COMUNA 07,1424.0,C1424BLS,-34.628207,-58.429515,109453.0
3444,MOLDES,3800.0,41897165,VTA,DTO,48,43,U$S 108.000,1890000,0,...,NaN,NaN,MOLDES 3800,SAAVEDRA,COMUNA 12,1429.0,C1429AFR,-34.549328,-58.471972,108000.0
2441,SOLER,4100.0,41880761,VTA,DTO,37,33,U$S 78.000,1365000,0,...,NaN,NaN,SOLER 4100,PALERMO,COMUNA 14,1425.0,C1425BWR,-34.590912,-58.418971,78000.0
6594,NaN,NaN,41962553,VTA,DTO,39,37,U$S 85.000,1487500,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85000.0


## Ejercicio 3  - Precio por metro cuadrado en dolares

Vamos a crear una columna de tipo numérico (PRECIOTEXTM) que tenga el precio del m2 en dolares calculado como el valor de PRECIOTEXT_CLEAN / M2

In [9]:
df['PRECIOTEXTM'] = df.apply(lambda x: x['PRECIOTEXTCLEAN'] / x['M2'], axis=1)

df.head(3)

,CALLE,NUMERO,ID_ZONAPRO,OPERACION,TIPO,M2,M2CUB,PRECIOTEXT,PRECIOARS,PRECIOARSM,...,NOTA,DIRECCION_NORMALIZADA,BARRIO,COMUNA,CODIGO_POSTAL,CODIGO_POSTAL_ARGENTINO,LATITUD,LONGITUD,PRECIOTEXTCLEAN,PRECIOTEXTM
0,GUATEMALA,5574.0,42408691,VTA,DTO,57,50,U$S 170.150,2977625,59553,...,NaN,GUATEMALA 5574,PALERMO,COMUNA 14,1425.0,C1425BVH,-34.580581,-58.431758,170150.0,2985.087719
1,ZAPATA,300.0,42408710,VTA,DTO,46,46,U$S 118.650,2076375,45139,...,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609,118650.0,2579.347826
2,ZAPATA,300.0,42518390,VTA,DTO,61,56,U$S 181.470,3175725,56709,...,NaN,ZAPATA 300,PALERMO,COMUNA 14,1426.0,C1426AED,-34.573870,-58.440609,181470.0,2974.918033


## Ejercicio 4  - Precio promedio del metro cuadrado en dolares por barrio

Calculemos el precio promedio del metro cuadrado en dolares por barrio usando groupby y pivot_table

¿Cuáles son los cinco barrios más caros?

In [10]:
precio_promedio = df.groupby('BARRIO')['PRECIOTEXTM'].mean()
a=precio_promedio.reset_index().sort_values('PRECIOTEXTM', ascending = False)
a.reset_index(drop=True)[:5]

,BARRIO,PRECIOTEXTM
0,PUERTO MADERO,5412.468738
1,PALERMO,3136.014984
2,BELGRANO,3110.065913
3,RETIRO,2941.052687
4,RECOLETA,2876.201544


In [11]:
b = df.pivot_table(index = "BARRIO", aggfunc={'PRECIOTEXTM':'mean'})
#En el index, ponemos el valor a partir del cual vamos a agrupar 
#Sobre los grupos que se arman a partir de INDEX, mirando el campo 'PRECIOTEXTM', aplica la funcion 'mean'
b_sorted = b.sort_values(by='PRECIOTEXTM', ascending = False)
b_sorted[:5]

,PRECIOTEXTM
BARRIO,
PUERTO MADERO,5412.468738
PALERMO,3136.014984
BELGRANO,3110.065913
RETIRO,2941.052687
RECOLETA,2876.201544


## Ejercicio 5  - groupby por campos numéricos (decilos)

Armemos los decilos de superficie de data usando el método `qcut` de numpy

Esto asocia una categoría a cada registro de data, donde el valor de esa categoría indica a qué decilo pertenece ese registro

Usando el método `groupby` calculemos la mediana del precio por m2 en dolares para cada uno de los decilos de superficie.

In [12]:
q_superficie = pd.qcut(df.M2, 10)
q_superficie

0         (54.0, 64.0]
1         (43.0, 48.0]
2         (54.0, 64.0]
3       (123.0, 730.0]
4         (35.0, 39.0]
             ...      
7559      (35.0, 39.0]
7560      (43.0, 48.0]
7561      (43.0, 48.0]
7562      (43.0, 48.0]
7563      (48.0, 54.0]
Name: M2, Length: 7564, dtype: category
Categories (10, interval[float64]): [(14.999, 35.0] < (35.0, 39.0] < (39.0, 43.0] < (43.0, 48.0] ... (64.0, 74.0] < (74.0, 88.0] < (88.0, 123.0] < (123.0, 730.0]]

In [14]:
mediana_m2 = df.groupby(q_superficie)['PRECIOTEXTM'].median()
mediana_m2.reset_index()

,M2,PRECIOTEXTM
0,"(14.999, 35.0]",2452.657143
1,"(35.0, 39.0]",2283.483483
2,"(39.0, 43.0]",2300.000000
3,"(43.0, 48.0]",2377.777778
4,"(48.0, 54.0]",2283.018868
5,"(54.0, 64.0]",2374.965268
6,"(64.0, 74.0]",2302.173913
7,"(74.0, 88.0]",2375.000000
8,"(88.0, 123.0]",2552.684117
9,"(123.0, 730.0]",2692.307692


## Ejercicio 6  - groupby por campos de tipo string

Usando el método `groupby` calculemos la mediana del precio por m2 en dolares para cada uno de los barrios de la ciudad.

In [17]:
df.groupby('BARRIO')['M2'].median()

BARRIO
AGRONOMIA             40.0
ALMAGRO               53.0
BALVANERA             55.0
BARRACAS              65.5
BELGRANO              74.0
BOCA                  59.5
BOEDO                 47.0
CABALLITO             54.0
CHACARITA             45.0
COGHLAN               74.5
COLEGIALES            70.0
CONSTITUCION          50.0
FLORES                71.0
FLORESTA              56.0
LINIERS               58.5
MATADEROS             75.0
MONSERRAT             39.5
MONTE CASTRO          47.0
NUEVA POMPEYA         35.5
NUÑEZ                 60.0
PALERMO               57.0
PARQUE AVELLANEDA     45.0
PARQUE CHACABUCO      45.0
PARQUE CHAS           61.5
PARQUE PATRICIOS      56.0
PATERNAL              77.0
PUERTO MADERO        117.0
RECOLETA              74.0
RETIRO                68.0
SAAVEDRA              54.5
SAN CRISTOBAL         52.0
SAN NICOLAS           46.0
SAN TELMO             53.0
VELEZ SARSFIELD       73.0
VERSALLES             70.5
VILLA CRESPO          42.0
VILLA DEL PARQUE     

In [18]:
df.pivot_table(index='BARRIO', aggfunc={'M2':'median'})

,M2
BARRIO,
AGRONOMIA,40.0
ALMAGRO,53.0
BALVANERA,55.0
BARRACAS,65.5
BELGRANO,74.0
BOCA,59.5
BOEDO,47.0
CABALLITO,54.0
CHACARITA,45.0


## Ejercicio 7  - groupby por condiciones custom

Definamos una máscara booleana que sea True para los registros cuyo precio en dolares es hasta 150.000

Agrupemos por este criterio, y calculemos el valor de la mediana de superficie para estos dos grupos (precio <= 150.000 dolares y precio > 150.000 dolares)

In [27]:
bool_mask = df['PRECIOTEXTCLEAN'] <= 150000


df.groupby(bool_mask)["M2"].median().reset_index()

,PRECIOTEXTCLEAN,M2
0,False,83
1,True,44


para los departamentos que valen mas de 150.000 usd, su mediana es de 83m2, mientras que para los departamentos que valen menos de 150.000 usd, su mediana es de 44m2